In [29]:
import pandas as pd

df = pd.read_csv('all_kindle_review.csv')

In [30]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [31]:
df = df[['reviewText','rating']] 

In [32]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [33]:
df.shape

(12000, 2)

In [34]:
df['rating'].dtype

dtype('int64')

In [35]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [36]:
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

In [37]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [38]:
df['reviewText']=df['reviewText'].str.lower()

In [39]:
import re
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    # 1. Remove URLs
    text = re.sub(r'(http|https|ftp|ssh)://\S+', ' ', text)
    
    # 2. Remove non-alphanumeric characters (keep spaces)
    text = re.sub('[^a-z A-Z 0-9 ]+', ' ', text)
    
    # 3. Lowercase
    text = text.lower()
    
    # 4. Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    # 5. Remove extra spaces
    text = " ".join(text.split())
    
    return text

# Apply once
df['reviewText'] = df['reviewText'].apply(clean_text)


In [41]:
df['reviewText']

0        jace rankin may short nothing mess man hauled ...
1        great short read want put read one sitting sex...
2        start saying first four books expecting 34 con...
3        aggie angela lansbury carries pocketbooks inst...
4        expect type book library pleased find price right
                               ...                        
11995    valentine cupid vampire jena ian another vampi...
11996    read seven books series apocalyptic adventure ...
11997    book really cuppa situation man capturing woma...
11998    tried use charge kindle even register charging...
11999    taking instruction look often hidden world sex...
Name: reviewText, Length: 12000, dtype: object

In [42]:
df

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four books expecting 34 con...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1
...,...,...
11995,valentine cupid vampire jena ian another vampi...,1
11996,read seven books series apocalyptic adventure ...,1
11997,book really cuppa situation man capturing woma...,1
11998,tried use charge kindle even register charging...,0


In [50]:
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()

# def lemmatize_text(text):
#     return " ".join([lemma.lemmatize(word, pos='v') for word in text.split()])

# df['reviewText'] = df['reviewText'].apply(lemmatize_text)

df['reviewText'] = [lemma.lemmatize(word,pos='v') for word in df['reviewText']]

In [52]:
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man haul sa...,1
1,great short read want put read one sit sex sce...,1
2,start say first four book expect 34 conclude 3...,1
3,aggie angela lansbury carry pocketbooks instea...,1
4,expect type book library please find price right,1


In [53]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df['reviewText'], df['rating'] , test_size=0.2,random_state=42)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=100)

X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()


In [55]:
X_train

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 2, 1],
       [2, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [56]:
from sklearn.naive_bayes import MultinomialNB

mn = MultinomialNB()

model = mn.fit(X_train,y_train)

In [57]:
y_pred = model.predict(X_test)



In [59]:
from sklearn.metrics import classification_report,accuracy_score

print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.7354166666666667
              precision    recall  f1-score   support

           0       0.63      0.49      0.56       803
           1       0.77      0.86      0.81      1597

    accuracy                           0.74      2400
   macro avg       0.70      0.68      0.68      2400
weighted avg       0.73      0.74      0.73      2400

